In [1]:
import glob, h5py, pickle, cv2, re
from pandas import read_csv
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import mat73
from IPython.display import clear_output
from scipy.io import loadmat


width = 128
height = 128
channels = 3

num_class = 10
sub = 'ASL_' + str(num_class) + '_vid_skel_mDsim'
filename = '/mnt/HDD04/ICASSP_2023/Scripts/datasets/'+sub+'.pkl'
print('File to be created: ' + filename)

2022-10-10 15:07:31.852160: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-10 15:07:32.042049: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-10 15:07:32.796740: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/emre/anaconda3/envs/emre_venv/lib/python3.9/site-packages/cv2/../../lib64:
2022-10-10 15:07:32.796840: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvi

File to be created: /mnt/HDD04/ICASSP_2023/Scripts/datasets/ASL_10_vid_skel_mDsim.pkl


In [116]:
env_path = '/mnt/HDD04/ICASSP_2023/Data/envelopes_sim/'
env_threshold = 2e3
vid_path = '/mnt/HDD04/Gallaudet_exp1_fusion_data/videos/'
skel_path = env_path.replace('envelopes_sim', 'skeletons_mat')
md_path = env_path.replace('envelopes_sim', 'microDoppler_sim')

env_files = glob.glob(env_path + '*txt')
print('Num. env files:', len(env_files))

Num. env files: 2240


In [105]:
import itertools
clas_dict = dict()
for e in env_files:
    fname = e.split('/')[-1]
    underscore_idx = [m.start() for m in re.finditer('_', fname)]
    word_id = fname.find('word')
    clas = fname[word_id+4:underscore_idx[-1]]
    if clas not in clas_dict:
        clas_dict[clas] = 1
    else:
        clas_dict[clas] += 1
sorted_clas_dict = {k: v for k, v in sorted(clas_dict.items(), key=lambda item: item[1])}
reverse =dict(reversed(list(sorted_clas_dict.items())))
# select the ones with most samples
sorted_clas_dict = dict(itertools.islice(reverse.items(), num_class))
print(sorted_clas_dict)
select_classes = sorted_clas_dict.keys()
print('select_classes:', select_classes)

{'125': 30, '121': 30, '122': 30, '123': 25, '117': 25, '119': 25, '131': 25, '128': 25, '129': 25, '118': 25}
select_classes: dict_keys(['125', '121', '122', '123', '117', '119', '131', '128', '129', '118'])


In [106]:
new_classes = {'125': 0, '121': 1, '122': 2, '123': 3, '117': 4,
               '119': 5, '131': 6, '128': 7, '129': 8, '118': 9}

In [107]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return np.array(dataframe.values)

In [108]:
def FrameCapture(path): 
      
    # Path to video file 
    vidObj = cv2.VideoCapture(path) 
    video_length = int(vidObj.get(cv2.CAP_PROP_FRAME_COUNT))
#     print ("Number of frames: ", video_length)
    # Used as counter variable 
    count = 0
    success, frame = vidObj.read() 
    # checks whether frames were extracted 
#     success = 1
    frames = []
    while success: 
  
        resized = cv2.resize(frame, (width,height), interpolation = cv2.INTER_LINEAR)
        success, frame = vidObj.read() 
#         print('Read a new frame: ', success)
        frames.append(resized) # .resized(width,height)
        count += 1
        if (count > (video_length-1)):
            break
    result = np.array([frames[i] for i in range(len(frames))])
    return result

In [109]:
train_files, test_files = train_test_split(env_files, test_size=0.2, random_state=1)

In [117]:
train_images = []
train_vids = []
train_skels = []
train_labels = []

for i, e in enumerate(train_files):
    
    fname = e.split('/')[-1]
    underscore_idx = [m.start() for m in re.finditer('_', fname)]
    word_id = fname.find('word')
    clas = fname[word_id+4:underscore_idx[-1]]
    
    if clas not in select_classes:
        continue
        
    # read envelope
    env = load_file(e)
    if sum(env[0]) < env_threshold:
        continue
        
    # read video
    vid_name = vid_path + fname.replace('txt', 'avi')
    vid = FrameCapture(vid_name)
    num_frames = vid.shape[0]
    train_vids.append(np.array(vid))
    
    # read skeleton
    skel_name = skel_path + fname.replace('txt', 'mat')
    skel = loadmat(skel_name)['hands']
    skel = np.transpose(skel, [2, 1, 0])
    skel = skel[3:]
    
    # pad missing frames
    if skel.shape[0] < num_frames:
        skel = np.concatenate([skel, np.tile(np.expand_dims(skel[0], 0), [num_frames - skel.shape[0], 1, 1])])
    train_skels.append(skel)
    
    # read microDoppler
    md_name = md_path + fname.replace('txt', 'png')
    img = cv2.imread(md_name)
    img = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC) # resize to (128,128)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
    clear_output(wait=True)
    print('Processing training file: '+ str(i+1) + '/' + str(len(train_files)))
    train_images.append(img)
    train_labels.append(new_classes[clas])

train_labels = to_categorical(np.array(train_labels, dtype=object), num_class)
train_images = np.swapaxes(np.array(train_images, dtype=object), 1, 2).reshape(len(train_images), width, height, channels)/255.
train_vids = np.array(train_vids, dtype=object)/255.
train_skels = np.array(train_skels, dtype=object)
print(train_images.shape)
print(train_skels.shape)
print(train_labels.shape)
print(train_vids.shape)

Processing training file: 1783/1792
(210, 128, 128, 3)
(210,)
(210, 10)
(210,)


In [118]:
test_images = []
test_vids = []
test_skels = []
test_labels = []

for i, e in enumerate(test_files):
    
    fname = e.split('/')[-1]
    underscore_idx = [m.start() for m in re.finditer('_', fname)]
    word_id = fname.find('word')
    clas = fname[word_id+4:underscore_idx[-1]]
    
    if clas not in select_classes:
        continue
    
    # read envelope
    env = load_file(e)
    if sum(env[0]) < env_threshold:
        continue
        
    # read video
    vid_name = vid_path + fname.replace('txt', 'avi')
    vid = FrameCapture(vid_name)
    num_frames = vid.shape[0]
    test_vids.append(np.array(vid))
    
    # read skeleton
    skel_name = skel_path + fname.replace('txt', 'mat')
    skel = loadmat(skel_name)['hands']
    skel = np.transpose(skel, [2, 1, 0])
    skel = skel[3:]
    
    # pad missing frames
    if skel.shape[0] < num_frames:
        skel = np.concatenate([skel, np.tile(np.expand_dims(skel[0], 0), [num_frames - skel.shape[0], 1, 1])])
    test_skels.append(skel)
    
    # read microDoppler
    md_name = md_path + fname.replace('txt', 'png')
    img = cv2.imread(md_name)
    img = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC) # resize to (128,128)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
    clear_output(wait=True)
    print('Processing training file: '+ str(i+1) + '/' + str(len(test_files)))
    test_images.append(img)
    test_labels.append(new_classes[clas])

test_labels = to_categorical(np.array(test_labels, dtype=object), num_class)
test_images = np.swapaxes(np.array(test_images, dtype=object), 1, 2).reshape(len(test_images), width, height, channels)/255.
test_vids = np.array(test_vids, dtype=object)/255.
test_skels = np.array(test_skels, dtype=object)
print(test_images.shape)
print(test_skels.shape)
print(test_labels.shape)
print(test_vids.shape)

Processing training file: 437/448
(55, 128, 128, 3)
(55,)
(55, 10)
(55,)


In [119]:
data = [train_images, train_vids, train_skels, train_labels, 
        test_images, test_vids, test_skels, test_labels]
with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(data, output, pickle.HIGHEST_PROTOCOL)
print(filename+' created.')

/mnt/HDD04/ICASSP_2023/Scripts/datasets/ASL_10_vid_skel_mDsim.pkl created.
